# OPTIMIZANDO CODIGOS PARA ALTA PERFORMANCE

## TIPOS DEFINIDOS EM FUNÇÕES

Os tipos são muito importantes em JULIA. Um tipo mal definido pode atrasar e muito o desempenho de uma função/codigo. A melhor forma de obter um bom desempenho é casar o tipo do dado a ser trabalhado com o tipo que a função vai calcular.
No exemplo abaixo, temos uma função que soma elementos de um vetor. No primeiro teste a função soma_vetor1 recebe um vetor de qualquer tipo e trabalha com "soma" do tipo Float (soma = 0.0). No segundo teste a mesma função e vetores, porém a variável soma é do tipo inteiro.

In [36]:
# o comando i in x faz a variável i percorrer todo o vetor x
# pode ser utilizado for i = 1:length(x)

function soma_vetor1(x)
    soma = 0.0
    for i in x
        soma = soma + i
    end
    return soma
end

soma_vetor1 (generic function with 1 method)

In [37]:
x1 = collect(0:0.0001:5000);    # Array{Float64,1},
x2 = linspace(0,5000,50000001); # LinSpace{Float64}
x3 = 0:0.0001:5000;             # FloatRange{Float64}

0.0:0.0001:5000.0

In [38]:
# confirmandoo tipo
typeof(x1) , typeof(x2) , typeof(x3)

(Array{Float64,1},LinSpace{Float64},FloatRange{Float64})

In [39]:
@timev soma_vetor1(x1)

  0.064134 seconds (1.18 k allocations: 50.950 KB)
elapsed time (ns): 64134348
bytes allocated:   52173
pool allocs:       1178


1.2500000249999998e11

In [40]:
@timev soma_vetor1(x2)

  0.239496 seconds (1.63 k allocations: 66.964 KB)
elapsed time (ns): 239496126
bytes allocated:   68571
pool allocs:       1634


1.2500000249999998e11

In [41]:
@timev soma_vetor1(x3)

  0.237606 seconds (1.25 k allocations: 53.471 KB)
elapsed time (ns): 237605986
bytes allocated:   54754
pool allocs:       1255


1.2500000249999998e11

O tipo Array{Float64,1} apresentou melhor desempenho de tempo.

A função agora modificada. Note que "soma" agora é do tipo inteiro, mas está recebendo um vetor do tipo Float

In [42]:
# o comando i in x faz a variável i percorrer todo o vetor x
# pode ser utilizado for i = 1:length(x)

function soma_vetor2(x)
    soma = 0             # so esse datalhe ja é suficiente para reduzir o desempenho
    for i in x
        soma = soma + i
    end
    return soma
end

soma_vetor2 (generic function with 1 method)

In [43]:
@timev soma_vetor2(x1)

  1.391655 seconds (150.00 M allocations: 2.235 GB, 25.19% gc time)
elapsed time (ns): 1391655209
gc time (ns):      350588051
bytes allocated:   2400082765
pool allocs:       150001826
GC pauses:         104
full collections:  2


1.2500000249999998e11

In [44]:
@timev soma_vetor2(x2)

  1.285765 seconds (150.00 M allocations: 2.235 GB, 10.36% gc time)
elapsed time (ns): 1285765089
gc time (ns):      133161043
bytes allocated:   2400082823
pool allocs:       150001983
GC pauses:         105


1.2500000249999998e11

In [45]:
@timev soma_vetor2(x3)

  1.235247 seconds (150.00 M allocations: 2.235 GB, 10.69% gc time)
elapsed time (ns): 1235247288
gc time (ns):      132010801
bytes allocated:   2400068478
pool allocs:       150001603
GC pauses:         105


1.2500000249999998e11

Veja que todos os testes apresentaram resultados muito ruins em função do tipo de "soma" ser inteiro.

## FUNÇÕES ANÔNIMAS

Funções anônimas (funções sem nome) são funções no qual não há nome para defini-las. Um bom exemplo são as funções o **map()** e em compreensões de lista. Não precisam usar o SymPy ou a forma de função genérica (funcao(variavel) = expressao). O uso principal para funções anônimas é passá-los para funções que assumem outras funções como argumentos. Em geral são mais lentas que as funções genéricas ou simbólicas. 

In [54]:
using FastAnonymous

INFO: Precompiling module FastAnonymous.
 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in symbol(::String, ::Vararg{String,N}) at ./deprecated.jl:30
 in include_from_node1(::String) at ./loading.jl:488 (repeats 2 times)
 in macro expansion; at ./none:2 [inlined]
 in anonymous at ./<missing>:?
 in eval(::Module, ::Any) at ./boot.jl:234
 in process_options(::Base.JLOptions) at ./client.jl:239
 in _start() at ./client.jl:318
while loading /home/jmarcellopereira/.julia/v0.5/Debug/src/AST.jl, in expression starting on line 62
ERROR: LoadError: LoadError: UndefVarError: SymbolNode not defined
 in include_from_node1(::String) at ./loading.jl:488 (repeats 2 times)
 in macro expansion; at ./none:2 [inlined]
 in anonymous at ./<missing>:?
 in eval(::Module, ::Any) at ./boot.jl:234
 in process_options(::Base.JLOptions) at ./client.jl:239
 in _start() at ./client.jl:318
while loading /home/jmarcellopereira/.julia/v0.5/Debug/src/Analysis.jl, in expression starting on line 51
while loading 

LoadError: LoadError: Failed to precompile FastAnonymous to /home/jmarcellopereira/.julia/lib/v0.5/FastAnonymous.ji.
while loading In[54], in expression starting on line 1

In [52]:
@timev map((w -> w^2),rand(10000,10000));

  6.735775 seconds (28.16 k allocations: 1.491 GB, 1.64% gc time)
elapsed time (ns): 6735774548
gc time (ns):      110608408
bytes allocated:   1601345693
pool allocs:       28161
malloc() calls:    2
GC pauses:         2
full collections:  2


A mesma função so que agora usando a macro @anon do pacote FastAnonymous

In [55]:
@timev map(@anon(w -> w^2),rand(10000,10000));

LoadError: LoadError: UndefVarError: @anon not defined
while loading In[55], in expression starting on line 1

## NÃO USE VARIÁVEIS GLOBAIS EM FUNÇÕES

Variáveis globais são uma tentação na vida de qualquer programador. Porém...

In [9]:
# função calcula a soma a partir de uma variável global X
# o comando i in x faz a variável i percorrer todo o vetor x
# pode ser utilizado for i = 1:length(x)

function soma_vetorG()
    soma = 0.0             # cuidado! soma é tipo float, se não piora mais ainda
    for i in x
        soma = soma +  i
    end
    return soma
end

soma_vetorG (generic function with 1 method)

In [10]:
x = collect(0:0.0001:5000);

In [11]:
@timev soma_vetorG()

  6.325527 seconds (200.00 M allocations: 3.725 GB, 7.71% gc time)
elapsed time (ns): 6325526952
gc time (ns):      488000051
bytes allocated:   4000095581
pool allocs:       200001371
GC pauses:         174
full collections:  1


1.2500000249999998e11

A mesma função só que usando variável local passada para função

In [12]:
# função calcula a soma a partir de uma variável local X passada como valor
# o comando i in x faz a variável i percorrer todo o vetor x
# pode ser utilizado for i = 1:length(x)

function soma_vetorL(x)
    soma = 0.0             # cuidado! soma é tipo float
    for i in x
        soma = soma + i
    end
    return soma
end

soma_vetorL (generic function with 1 method)

In [13]:
x = collect(0:0.0001:5000);

In [15]:
@timev soma_vetorL(x)

  0.057018 seconds (5 allocations: 176 bytes)
elapsed time (ns): 57018320
bytes allocated:   176
pool allocs:       5


1.2500000249999998e11

Diferença absurda de tempo de alocação de memória entre o primeiro caso e segundo caso. 

## UTILIZE FUNÇÕES PARA SUBSTITUIR BLOCOS DE CÓDIGO

Bloco de código que utiliza variáveis globais para calcular a soma todos os elementos de um vetor

In [56]:
tic()
x = collect(0:0.0001:5000)
soma = 0.0             # cuidado! soma é tipo float
for i in x
    soma = soma + i
end

display(soma)

toc()

1.2500000249999998e11

elapsed time: 6.664979493 seconds


6.664979493

Função que substitui o procedimento acima e usa variáveis locais

In [57]:
# função calcula a soma a partir de uma variável local X passada como valor
# o comando i in x faz a variável i percorrer todo o vetor x
# pode ser utilizado for i = 1:length(x)

function soma_vetorL(x)
    soma = 0.0             # cuidado! soma é tipo float
    for i in x
        soma = soma + i
    end
    return soma
end

soma_vetorL (generic function with 1 method)

In [59]:
@timev soma_vetorL(collect(0:0.0001:5000))

  0.621667 seconds (9 allocations: 381.470 MB, 2.99% gc time)
elapsed time (ns): 621667059
gc time (ns):      18599932
bytes allocated:   400000416
pool allocs:       8
malloc() calls:    1
GC pauses:         1
full collections:  1


1.2500000249999998e11

Veja que a diferença de tempo é absurda!

## QUAL A MELHOR: FUNÇÕES GENÉRICAS, SIMBÓLICAS OU ANÔNIMAS?

## OTIMIZANDO UBUNTU E DERIVADOS

O Ubuntu e outros sistemas linux, possuem uma configuração de **"Concurrency"** que permite habilitar todos os núcleos, isso faz com que o Boot do seu sistema seja mais rápido que o normal e pode melhorar o desempenho do processamento numérico da Julia. Para habilitar o recurso, digite no shell:

```bash
sudo gedit /etc/init.d/rc
```
Para evitar problemas, salve o arquivo como **rc.back**, caso ocorra algum erro ou instabilidade do sistema. Procure as linhas **CONCURRENCY=makefile** e descomente, e comente a linha **#CONCURRENCY=none**. Veja o código abaixo:

    # Check if we are able to use make like booting.  It require the
    # insserv package to be enabled. Boot concurrency also requires
    # startpar to be installed.
    #
    CONCURRENCY=makefile # descomente a linha
    # disable startpar, incompatible with "task" upstart jobs
    #CONCURRENCY=none   # comente a linha
    
Reinicie e veja a diferença :)

## PARALELISMO 

### TRABALHAR COM CORES
Para adicionar cores utilize:
```julia
addproces(n)
```
Sendo **n** o número de cores que deseja criar as instâncias. Uma boa dica é usar ``addroces()`` dessa forma Julia cria automaticamente o número de cores que melhor se ajusta à máquina.

In [8]:
# visualizar o numero de cores do processador físico
CPU_CORES

  likely near In[8]:2


4

In [9]:
# verificar o número de cores disponíveis e ID
procs()

1-element Array{Int64,1}:
 1

In [10]:
# adicionar 2 cores
addprocs(2)
procs()

3-element Array{Int64,1}:
 1
 2
 3

In [12]:
# remover cores rmprocs(ID_CORE)
# para remover mais de um core use rmprocess([vetor_IDCORE])
rmprocs(4)
procs()

3-element Array{Int64,1}:
 1
 2
 3

### PARALLEL FOR

@parallel

In [13]:
# verificar o número de processadores
procs()

3-element Array{Int64,1}:
 1
 2
 3

In [14]:
# Função Soma.
# Laço for de uma soma de 1 a 1000000000
# de números aleatórios

function serial_add()
    s = 0.0
    for i = 1:1000000000
        s = s + randn()
    end
    return s
end 

serial_add (generic function with 1 method)

In [15]:
@timev serial_add()

  5.711486 seconds (20.87 k allocations: 950.084 KB, 0.13% gc time)
elapsed time (ns): 5711486252
gc time (ns):      7411350
bytes allocated:   972886
pool allocs:       20871
non-pool GC allocs:1
GC pauses:         1


7956.158837498818

In [16]:
# função paralela

function parallel_add()
    return @parallel (+) for i = 1:1000000000
                            randn()
                        end
end

parallel_add (generic function with 1 method)

In [17]:
# execute mais de uma vez
@timev parallel_add()

  4.258522 seconds (553.37 k allocations: 23.324 MB, 0.27% gc time)
elapsed time (ns): 4258521577
gc time (ns):      11551935
bytes allocated:   24456981
pool allocs:       553355
non-pool GC allocs:16
GC pauses:         2


-24614.67296161454

### ParallelAccelerator

ParallelAccelerator é um pacote para acelerar programas de Julia voltados principalmente para o cálculo de matrizes e vetores. Com a macro `@acc` **ParallelAccelerator** compila partes do programa e elimina automaticamente funções desnecessárias e também paraleliza e vetoriza muitas operações de dados em paralelo. Além de acelerar o cálculo, `@acc` optimiza o consumo de memória, logo é muito útil para grandes matrizes. 

ParallelAccelerator é parte do projeto de script de alto desempenho (HPS) da Intel Labs. 



In [19]:
Pkg.add("ParallelAccelerator")

INFO: Nothing to be done
INFO: METADATA is out-of-date — you may not have the latest version of ParallelAccelerator
INFO: Use `Pkg.update()` to get the latest versions of your packages


In [20]:
h(x)  = sum(x.^2 + x.^3 + x.^5 + x.^7 + x.^9 + x.^11) 

h (generic function with 1 method)

In [21]:
@timev h(rand(5000,5000))

 12.963267 seconds (78.77 k allocations: 2.239 GB, 3.64% gc time)
elapsed time (ns): 12963266895
gc time (ns):      471809417
bytes allocated:   2403620194
pool allocs:       78758
malloc() calls:    12
GC pauses:         12
full collections:  7


2.645991633212997e7

In [22]:
# use o garbage collector para remover o lixo da memória
gc()

In [23]:
using ParallelAccelerator

In [24]:
@acc g(x) = sum(x.^2 + x.^3 + x.^5 + x.^7 + x.^9 + x.^11) 

g (generic function with 1 method)

In [27]:
# execute sempre mais de uma vez para obter o melhor resultado
@timev g(rand(5000,5000));

  3.682378 seconds (50 allocations: 190.737 MB, 0.08% gc time)
elapsed time (ns): 3682378165
gc time (ns):      3077335
bytes allocated:   200001904
pool allocs:       49
malloc() calls:    1
GC pauses:         1


In [25]:
gc()

### DistributedArrays

In [26]:
Pkg.add("DistributedArrays")

INFO: Cloning cache of DistributedArrays from https://github.com/JuliaParallel/DistributedArrays.jl.git
INFO: Installing DistributedArrays v0.3.0
INFO: Package database updated
INFO: METADATA is out-of-date — you may not have the latest version of DistributedArrays
INFO: Use `Pkg.update()` to get the latest versions of your packages


In [27]:
function array()
    return ([(i^2+j^3)/log(abs(i*j)) for i = 1:20000, j = 1:10000]);
end

array (generic function with 1 method)

In [28]:
@timev array();

  8.963986 seconds (39.84 k allocations: 1.492 GB, 0.21% gc time)
elapsed time (ns): 8963985626
gc time (ns):      18543173
bytes allocated:   1601786100
pool allocs:       39841
malloc() calls:    1
GC pauses:         1
full collections:  1


In [29]:
# limpar memória
gc()

Antes de executar o código é necesário verificar se há outros cores disponíveis. Caso exista somente 1, é necessário adicionar novos cores e vincular o `DistributedArrays` ao macro `@everywhere` seguindo a ordem:

    1º adicione os cores addprocs(n*)
    2º use @everywhere using DistributedArrays
    3º execute o código com o macro @Darray 
**n** corresponde ao número de cores. Utilizando `addprocs()`(o mesmo que usar ``addprocs(CPU_CORES)``) Julia cria automaticamente o número de cores que melhor se ajustam à máquina. Sempre execute uma segunda vez o código, a primeira tem péssimo desempenho.

Caso execute novamente `addprocs(n)` e depois `@Darray`, os cores novos serão eliminados da lista de processos e será utilizado somente os cores vinculados ao macro `@everywhere`. 

In [30]:
# visualizar o numero de cores do processador físico
CPU_CORES

  likely near In[30]:2


4

In [31]:
# verificar o número de cores disponíveis e ID
procs()

3-element Array{Int64,1}:
 1
 2
 3

In [32]:
@everywhere using DistributedArrays

INFO: Precompiling module DistributedArrays.


In [33]:
function Darray()
    return @DArray ([(i^2+j^3)/log(abs(i*j)) for i = 1:20000, j = 1:10000]);
end

Darray (generic function with 1 method)

In [34]:
# a primeira execução é ruim. Execute uma segunda vez
@time  Darray();

  5.989773 seconds (1.14 M allocations: 47.956 MB, 0.31% gc time)


In [35]:
gc()